In [40]:
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [3]:
loader = WebBaseLoader(
    [
        'https://python.langchain.com/docs/introduction/',
        'https://docs.smith.langchain.com/',
        'https://www.investopedia.com/terms/a/agent.asp',

    ]
)

docs = loader.load()

In [4]:
docs

[Document(metadata={'source': 'https://python.langchain.com/docs/introduction/', 'title': 'Introduction | \uf8ffü¶úÔ∏è\uf8ffüîó LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='\n\n\n\n\nIntroduction | \uf8ffü¶úÔ∏è\uf8ffüîó LangChain\n\n\n\n\n\n\nSkip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1\uf8ffüí¨SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augmented Generation (RAG) App: Part 2Build an Extraction ChainBuild an AgentTaggingBuild a Retrieval Augmented Generation (RAG) App: Part 1Build a semantic search engineBuild a Question/Answering system

In [5]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
final_documents = splitter.split_documents(docs)

In [6]:
final_documents

[Document(metadata={'source': 'https://python.langchain.com/docs/introduction/', 'title': 'Introduction | \uf8ffü¶úÔ∏è\uf8ffüîó LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='Introduction | \uf8ffü¶úÔ∏è\uf8ffüîó LangChain'),
 Document(metadata={'source': 'https://python.langchain.com/docs/introduction/', 'title': 'Introduction | \uf8ffü¶úÔ∏è\uf8ffüîó LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='Skip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1\uf8ffüí¨SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with cha

In [8]:
embedding = OllamaEmbeddings(model='nomic-embed-text')

vectorstore = Chroma.from_documents(documents=final_documents, embedding=embedding)

In [9]:
vectorstore.similarity_search('what is ai agent')

[Document(metadata={'description': 'An agent is a person who is empowered to act on behalf of another. Read about different agent types, such as real estate, insurance, and business agents.', 'language': 'en', 'source': 'https://www.investopedia.com/terms/a/agent.asp', 'title': 'What Is an Agent? Definition, Types of Agents, and Examples'}, page_content='Learn about our \neditorial policies\n\n\n\n\n \n\n\n\n\n\n \n Investopedia / Joules Garcia\n\n\n\n\nClose\n\n\n\n  What Is an Agent?  \n An agent, in legal terminology, is a person who has been legally empowered to act on behalf of another person or an entity. An agent may be employed to represent a client in negotiations and other dealings with third parties. The agent may be given decision-making authority.\n\n\n Two common types of agents are attorneys, who represent their clients in legal matters, and stockbrokers, who are hired by investors to make investment decisions for them. The person represented by the agent in these scenar

In [10]:
retriever = vectorstore.as_retriever(
    search_kwargs = {'k' : 5}
)

In [11]:
retriever.invoke('what is agent')

[Document(metadata={'description': 'An agent is a person who is empowered to act on behalf of another. Read about different agent types, such as real estate, insurance, and business agents.', 'language': 'en', 'source': 'https://www.investopedia.com/terms/a/agent.asp', 'title': 'What Is an Agent? Definition, Types of Agents, and Examples'}, page_content='Learn about our \neditorial policies\n\n\n\n\n \n\n\n\n\n\n \n Investopedia / Joules Garcia\n\n\n\n\nClose\n\n\n\n  What Is an Agent?  \n An agent, in legal terminology, is a person who has been legally empowered to act on behalf of another person or an entity. An agent may be employed to represent a client in negotiations and other dealings with third parties. The agent may be given decision-making authority.\n\n\n Two common types of agents are attorneys, who represent their clients in legal matters, and stockbrokers, who are hired by investors to make investment decisions for them. The person represented by the agent in these scenar

In [14]:
llm = ChatOllama(model='deepseek-r1:1.5b')

In [46]:
template = """
Answer the questions based on the provided context only.
Please provide the most accurate response based on the question
context>
{context}
<context>

Questions:{input}
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ('system', 'You are an helpful AI assistant'),
        ('user', template)
    ]
)

In [47]:
chain = (
    {'context' : retriever, 'input' : RunnablePassthrough()}
    | prompt
    | llm 
    | StrOutputParser()
)

In [48]:
chain.invoke('what is Ai agents')

'<think>\nOkay, the user asked about AI agents. Hmm, I should start by understanding what an agent is in general from the context provided.\n\nLooking at the documents, it seems like the main definition given is that an agent is someone empowered to act on behalf of another. That means agents have a role to play based on another person\'s interest or need.\n\nThen, there are some examples of different types of agents. Real estate agents, insurance agents, and business agents each serve specific purposes. For instance, real estate agents help buy and sell homes, while insurance agents manage risks related to life, property, etc.\n\nNow, the user might be interested in knowing more about AI agents specifically. I need to check if there\'s a section that mentions AI agents or if they fall under other categories like software agents or legal agents. From what I see, real estate and insurance agents are mentioned, but AI isn\'t listed explicitly. Maybe it\'s part of the broader category of 